## Módulo: Analytics Engineering
    
## Projeto: Engenharia de Dados e Garantia de Qualidade no Conjunto de Dados do Airbnb no Rio de Janeiro.

#### Introdução à Base de Dados do Airbnb

O conjunto de dados "Inside Airbnb", disponível no website "http://insideairbnb.com/", é uma valiosa fonte de informações sobre listagens de hospedagem, avaliações de hóspedes e disponibilidade de calendário em várias cidades ao redor do mundo, incluindo o Rio de Janeiro. Antes de prosseguirmos com a engenharia de dados, é importante entender os principais componentes deste conjunto de dados:

1. **Listing (Listagem):** Este conjunto de dados contém informações detalhadas sobre as propriedades listadas no Airbnb. Cada registro representa uma listagem individual e inclui informações como o tipo de propriedade, preço, localização, número de quartos, comodidades oferecidas e muito mais.

2. **Reviews (Avaliações):** O conjunto de dados de avaliações contém informações sobre as avaliações feitas por hóspedes que ficaram nas propriedades listadas. Ele inclui dados como a data da avaliação, o identificador da propriedade, os comentários escritos pelos hóspedes, e outras informações. 

3. **Calendar (Calendário):** Este conjunto de dados contém informações sobre a disponibilidade das propriedades ao longo do tempo. Ele lista as datas em que as propriedades estão disponíveis para reserva, bem como os preços para cada data.

O dicionário dos dados também está disponível no website: "http://insideairbnb.com/".

#### Passos do Projeto

1. **Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

<br>

2. **Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

<br>

3. **Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

<br>

4. **Testes de Qualidade - Camada Silver:**
   - Utilize a biblioteca Great Expectations para criar testes de qualidade automatizados que verifiquem as expectativas definidas para os dados da camada "silver".
   - Desenvolva testes que assegurem que os dados da camada "silver" atendam às regras de negócios e aos requisitos de qualidade.

<br>

5. **Transformação de Dados com dbt - Camada Silver:**
   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

<br>

6. **Armazenamento de Dados em PostgreSQL - Camada Silver:**
   - Armazene os dados da camada "silver" no mesmo banco de dados PostgreSQL.
   - Estabeleça conexões entre o dbt e o PostgreSQL para carregar os dados transformados da camada "silver" no banco.

<br>

7. **Validação de Expectativas com Great Expectations - Camada Silver:**
   - Implemente validações adicionais usando Great Expectations nas camadas de dados da camada "silver".
   - Monitore a qualidade dos dados da camada "silver" após cada transformação e ajuste os testes de acordo.

<br>

8. **Transformação de Dados com dbt - Camada Gold:**
   - Utilize o dbt para criar a camada "gold" de dados, aplicando agregações especializadas, como médias de preços por propriedade, por período, e outras agregações especializadas.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "gold" e automatize a execução das transformações.
   - Armazene os dados da camada "gold" no mesmo banco de dados PostgreSQL, mantendo a estrutura de dados otimizada para consultas analíticas.

<br>

 9. **Apresentação e Discussão:**
    - Apresente os resultados do projeto para a turma, enfatizando os aspectos de engenharia de dados, qualidade de dados e uso de ferramentas como dbt, Great Expectations e o armazenamento em um banco de dados PostgreSQL nas camadas "bronze", "silver" e "gold".


# Etapas

**Aquisição de Dados e Armazenamento de Dados em PostgreSQL - Camada Bronze**
   - Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
   - Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

##### Dependências

In [1]:
!pip install pandas
!pip install sqlalchemy
!pip install pymysql
!pip install psycopg2-binary

In [2]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

##### Importando os dados como DataFrame Pandas

In [3]:
calendar = pd.read_csv('data_csv/calendar.csv')
listings = pd.read_csv('data_csv/listings.csv')
reviews = pd.read_csv('data_csv/reviews.csv')

In [4]:
calendar.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


In [6]:
listings.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,17878,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,68997,Matthias,NaN,Copacabana,-22.96599,-43.17940,Entire home/apt,279,5,301,2023-09-11,1.87,1,265,25,NaN
1,25026,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,102840,Viviane,NaN,Copacabana,-22.97735,-43.19105,Entire home/apt,330,2,272,2023-09-07,1.68,1,203,24,NaN
2,35764,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,153691,Patricia Miranda & Paulo,NaN,Copacabana,-22.98107,-43.19136,Entire home/apt,192,3,446,2023-09-11,2.82,1,46,37,NaN
3,48305,Rental unit in Ipanema · ★4.74 · 6 bedrooms · ...,70933,Goitaca,NaN,Ipanema,-22.98591,-43.20302,Entire home/apt,3448,2,152,2023-09-10,0.99,9,306,30,NaN
4,48901,Rental unit in Rio · ★4.37 · 4 bedrooms · 5 be...,222884,Marcio,NaN,Copacabana,-22.96574,-43.17514,Entire home/apt,703,3,20,2023-09-10,0.20,1,307,12,NaN


In [17]:
reviews.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


##### Importando DataFrame para o PostgreSQL

In [10]:
user = 'postgres'
password = 'ada_postgres_1009'
host = 'database-ada-1009.clzgkgrmzmzh.us-east-1.rds.amazonaws.com'
port = '5432'
database = 'postgres'
schema = 'raw'

In [11]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

calendar

In [18]:
raw_calendar_table = 'danilo_freitas_raw_calendar'

In [ ]:
calendar.to_sql(name=raw_calendar_table, con=engine, if_exists='replace', index=False, schema=schema)


listings

In [19]:
raw_listings_table = 'danilo_freitas_raw_listings'

In [20]:
listings.to_sql(name=raw_listings_table, con=engine, if_exists='replace', index=False, schema=schema)

964

reviews

In [21]:
raw_reviews_table = 'danilo_freitas_raw_reviews'

In [22]:
reviews.to_sql(name=raw_reviews_table, con=engine, if_exists='replace', index=False, schema=schema)

307

**Data Clean - Camada Silver:**
   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".
   - Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.
   - Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

##### Leitura do raw data do banco de dados 

In [6]:
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "ada_postgres_1009",
    "host": "database-ada-1009.clzgkgrmzmzh.us-east-1.rds.amazonaws.com",  
    "port": "5432"  
}

In [7]:
conn = psycopg2.connect(**db_params)

##### calendar

In [9]:
calendar_raw = "SELECT * FROM raw.danilo_freitas_raw_calendar;"

In [10]:
calendar_raw_df = pd.read_sql_query(calendar_raw, conn)

/tmp/ipykernel_26692/1951227670.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  calendar_raw_df = pd.read_sql_query(calendar_raw, conn)


In [11]:
calendar_raw_df.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


listings

In [12]:
listings_raw = "SELECT * FROM raw.danilo_freitas_raw_listings;"

In [13]:
listings_raw_df = pd.read_sql_query(listings_raw, conn)

/tmp/ipykernel_26692/959584078.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  listings_raw_df = pd.read_sql_query(listings_raw, conn)


In [14]:
listings_raw_df.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,17878,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,68997,Matthias,None,Copacabana,-22.96599,-43.17940,Entire home/apt,279,5,301,2023-09-11,1.87,1,265,25,None
1,25026,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,102840,Viviane,None,Copacabana,-22.97735,-43.19105,Entire home/apt,330,2,272,2023-09-07,1.68,1,203,24,None
2,35764,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,153691,Patricia Miranda & Paulo,None,Copacabana,-22.98107,-43.19136,Entire home/apt,192,3,446,2023-09-11,2.82,1,46,37,None
3,48305,Rental unit in Ipanema · ★4.74 · 6 bedrooms · ...,70933,Goitaca,None,Ipanema,-22.98591,-43.20302,Entire home/apt,3448,2,152,2023-09-10,0.99,9,306,30,None
4,48901,Rental unit in Rio · ★4.37 · 4 bedrooms · 5 be...,222884,Marcio,None,Copacabana,-22.96574,-43.17514,Entire home/apt,703,3,20,2023-09-10,0.20,1,307,12,None


reviews

In [15]:
reviews_raw = "SELECT * FROM raw.danilo_freitas_raw_reviews;"

In [16]:
reviews_raw_df = pd.read_sql_query(reviews_raw, conn)

/tmp/ipykernel_26692/4107597973.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews_raw_df = pd.read_sql_query(reviews_raw, conn)


In [17]:
reviews_raw_df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


##### calendar

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".

In [18]:
calendar_raw_df.isnull()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...
2983995,False,False,False,False,False,False,False
2983996,False,False,False,False,False,False,False
2983997,False,False,False,False,False,False,False
2983998,False,False,False,False,False,False,False


In [19]:
num_nulos_por_coluna = calendar_raw_df.isnull().sum()
print(num_nulos_por_coluna)

listing_id        0
date              0
available         0
price             0
adjusted_price    0
minimum_nights    3
maximum_nights    3
dtype: int64


In [20]:
calendar_raw_df.drop_duplicates(inplace=True)

In [21]:
calendar_raw_df.duplicated()

0          False
1          False
2          False
3          False
4          False
           ...  
2983995    False
2983996    False
2983997    False
2983998    False
2983999    False
Length: 2984000, dtype: bool

In [22]:
calendar_raw_df.describe()

,listing_id,minimum_nights,maximum_nights
count,2.984000e+06,2.983997e+06,2.983997e+06
mean,1.013328e+07,6.695479e+00,6.906041e+02
std,6.270713e+06,2.605333e+01,5.177538e+02
min,1.787800e+04,1.000000e+00,1.000000e+00
25%,3.183573e+06,2.000000e+00,6.000000e+01
50%,1.120795e+07,3.000000e+00,1.125000e+03
75%,1.421395e+07,5.000000e+00,1.125000e+03
max,2.240569e+07,1.125000e+03,1.825000e+03


- Padronize e limpe os nomes das colunas, convertendo-os em um formato consistente.

In [23]:
calendar_raw_df.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


In [24]:
calendar_raw_df.dtypes

listing_id          int64
date               object
available          object
price              object
adjusted_price     object
minimum_nights    float64
maximum_nights    float64
dtype: object

In [25]:
calendar_raw_df["date"] = pd.to_datetime(calendar_raw_df['date'])

In [26]:
new_name_columns = ['id_listing', 'data', 'disponibilidade', 'preco', 'preco_ajustado', 'minimo_noites', 'maximo_noites']

In [27]:
calendar_raw_df.columns = new_name_columns

In [28]:
calendar_raw_df.head(5)

,id_listing,data,disponibilidade,preco,preco_ajustado,minimo_noites,maximo_noites
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


- Realize uma limpeza textual em campos, como descrições de propriedades, removendo caracteres especiais e erros de digitação.

In [29]:
calendar_raw_df.head(50)

,id_listing,data,disponibilidade,preco,preco_ajustado,minimo_noites,maximo_noites
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0
5,17878,2023-09-28,f,$290.00,$290.00,5.0,28.0
6,17878,2023-09-29,f,$290.00,$290.00,5.0,28.0
7,17878,2023-09-30,f,$290.00,$290.00,5.0,28.0
8,17878,2023-10-01,f,$290.00,$290.00,5.0,28.0
9,17878,2023-10-02,t,$290.00,$290.00,5.0,28.0


In [30]:
danilo_freitas_processing_calendar = "danilo_freitas_processing_calendar"
trusted = "trusted"

In [31]:
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "ada_postgres_1009",
    "host": "database-ada-1009.clzgkgrmzmzh.us-east-1.rds.amazonaws.com",  
    "port": "5432"  
}

In [32]:
engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

In [49]:
calendar.to_sql(danilo_freitas_processing_calendar, con=engine, schema=trusted, if_exists='replace', index=False)

: 

##### listings

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".

In [33]:
listings_raw_df.isnull()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
1,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
2,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
3,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31959,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,True
31960,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,True
31961,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,True
31962,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,True


In [34]:
num_nulos_por_coluna_listings = listings_raw_df.isnull().sum()
print(num_nulos_por_coluna_listings)

id                                    0
name                                  0
host_id                               0
host_name                             3
neighbourhood_group               31964
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                        8140
reviews_per_month                  8140
calculated_host_listings_count        0
availability_365                      0
number_of_reviews_ltm                 0
license                           31964
dtype: int64


In [35]:
listings_raw_df.drop_duplicates(inplace=True)

In [36]:
listings_raw_df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
31959    False
31960    False
31961    False
31962    False
31963    False
Length: 31964, dtype: bool

In [37]:
listings_raw_df.describe()

,id,host_id,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm
count,3.196400e+04,3.196400e+04,31964.000000,31964.000000,31964.000000,31964.000000,31964.000000,23824.000000,31964.000000,31964.000000,31964.000000
mean,3.514699e+17,1.663077e+08,-22.967574,-43.251361,819.913027,4.740395,19.938274,0.974190,8.839319,188.491960,7.017426
std,3.999653e+17,1.684315e+08,0.035182,0.100595,6640.141788,23.653482,40.676059,1.127053,23.515196,133.012558,11.436773
min,1.787800e+04,1.671000e+03,-23.073276,-43.723009,33.000000,1.000000,0.000000,0.010000,1.000000,0.000000,0.000000
25%,2.184739e+07,2.258631e+07,-22.984820,-43.310047,196.000000,1.000000,0.000000,0.170000,1.000000,72.000000,0.000000
50%,5.261496e+07,8.693849e+07,-22.972860,-43.195670,317.000000,2.000000,4.000000,0.570000,2.000000,174.000000,2.000000
75%,7.814279e+17,3.004090e+08,-22.956165,-43.185946,600.000000,3.000000,20.000000,1.380000,4.000000,335.000000,9.000000
max,9.855551e+17,5.379850e+08,-22.749690,-43.104400,562031.000000,1125.000000,618.000000,14.780000,163.000000,365.000000,133.000000


In [38]:
listings_raw_df.head(5)

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,17878,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,68997,Matthias,None,Copacabana,-22.96599,-43.17940,Entire home/apt,279,5,301,2023-09-11,1.87,1,265,25,None
1,25026,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,102840,Viviane,None,Copacabana,-22.97735,-43.19105,Entire home/apt,330,2,272,2023-09-07,1.68,1,203,24,None
2,35764,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,153691,Patricia Miranda & Paulo,None,Copacabana,-22.98107,-43.19136,Entire home/apt,192,3,446,2023-09-11,2.82,1,46,37,None
3,48305,Rental unit in Ipanema · ★4.74 · 6 bedrooms · ...,70933,Goitaca,None,Ipanema,-22.98591,-43.20302,Entire home/apt,3448,2,152,2023-09-10,0.99,9,306,30,None
4,48901,Rental unit in Rio · ★4.37 · 4 bedrooms · 5 be...,222884,Marcio,None,Copacabana,-22.96574,-43.17514,Entire home/apt,703,3,20,2023-09-10,0.20,1,307,12,None


In [39]:
listings_raw_df.dtypes

id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
number_of_reviews_ltm               int64
license                            object
dtype: object

In [40]:
new_name_columns_listings = ['id', 'nome', 'id_hospede', 'nome_hospede', 'grupo_bairro', 'bairro', 'latitude', 'longitude', 'tipo', 'preco', 'minimo_noites', 'numero_avaliacoes', 'ultima_avaliacao', 'avaliacoes_por_mes', 'numero_hospedes', 'disponibilidade_365', 'avaliacoes_ltm', 'licenca']

In [41]:
listings_raw_df.columns = new_name_columns_listings

In [42]:
listings_raw_df.head(5)

,id,nome,id_hospede,nome_hospede,grupo_bairro,bairro,latitude,longitude,tipo,preco,minimo_noites,numero_avaliacoes,ultima_avaliacao,avaliacoes_por_mes,numero_hospedes,disponibilidade_365,avaliacoes_ltm,licenca
0,17878,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,68997,Matthias,None,Copacabana,-22.96599,-43.17940,Entire home/apt,279,5,301,2023-09-11,1.87,1,265,25,None
1,25026,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,102840,Viviane,None,Copacabana,-22.97735,-43.19105,Entire home/apt,330,2,272,2023-09-07,1.68,1,203,24,None
2,35764,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,153691,Patricia Miranda & Paulo,None,Copacabana,-22.98107,-43.19136,Entire home/apt,192,3,446,2023-09-11,2.82,1,46,37,None
3,48305,Rental unit in Ipanema · ★4.74 · 6 bedrooms · ...,70933,Goitaca,None,Ipanema,-22.98591,-43.20302,Entire home/apt,3448,2,152,2023-09-10,0.99,9,306,30,None
4,48901,Rental unit in Rio · ★4.37 · 4 bedrooms · 5 be...,222884,Marcio,None,Copacabana,-22.96574,-43.17514,Entire home/apt,703,3,20,2023-09-10,0.20,1,307,12,None


In [43]:
listings_raw_df["ultima_avaliacao"] = pd.to_datetime(listings_raw_df["ultima_avaliacao"])

In [44]:
listings_raw_df.dtypes

id                              int64
nome                           object
id_hospede                      int64
nome_hospede                   object
grupo_bairro                   object
bairro                         object
latitude                      float64
longitude                     float64
tipo                           object
preco                           int64
minimo_noites                   int64
numero_avaliacoes               int64
ultima_avaliacao       datetime64[ns]
avaliacoes_por_mes            float64
numero_hospedes                 int64
disponibilidade_365             int64
avaliacoes_ltm                  int64
licenca                        object
dtype: object

##### reviews

   - Identifique e lide com valores ausentes, duplicatas e outliers nos dados brutos da camada "bronze".

In [45]:
reviews_raw_df.isnull()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False
...,...,...,...,...,...,...
637302,False,False,False,False,False,False
637303,False,False,False,False,False,False
637304,False,False,False,False,False,False
637305,False,False,False,False,False,False


In [46]:
num_nulos_por_coluna_reviews = reviews_raw_df.isnull().sum()
print(num_nulos_por_coluna_reviews)

listing_id        0
id                0
date              0
reviewer_id       0
reviewer_name     0
comments         19
dtype: int64


In [47]:
reviews_raw_df.drop_duplicates(inplace=True)

In [48]:
reviews_raw_df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
637302    False
637303    False
637304    False
637305    False
637306    False
Length: 637307, dtype: bool

In [49]:
reviews_raw_df.describe()

,listing_id,id,reviewer_id
count,6.373070e+05,6.373070e+05,6.373070e+05
mean,1.128101e+17,4.469503e+17,1.852193e+08
std,2.636175e+17,3.845078e+17,1.509693e+08
min,1.787800e+04,5.063600e+04,5.000000e+00
25%,9.642367e+06,5.381218e+08,5.029380e+07
50%,2.702873e+07,5.578677e+17,1.499738e+08
75%,4.870192e+07,8.100821e+17,3.004780e+08
max,9.825902e+17,9.862971e+17,5.381118e+08


In [50]:
reviews_raw_df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


In [51]:
reviews_raw_df.dtypes

listing_id        int64
id                int64
date             object
reviewer_id       int64
reviewer_name    object
comments         object
dtype: object

In [58]:
new_name_columns_reviews = ['id_listing', 'id', 'data', 'reviewer_id', 'reviewer_name','comentarios']

In [59]:
reviews_raw_df.columns = new_name_columns_reviews

In [60]:
reviews_raw_df.head(5)

,id_listing,id,data,reviewer_id,reviewer_name,comentarios
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


In [61]:
reviews_raw_df["data"] = pd.to_datetime(reviews_raw_df["data"])

In [62]:
reviews_raw_df.dtypes

id_listing                int64
id                        int64
data             datetime64[ns]
reviewer_id               int64
reviewer_name            object
comentarios              object
dtype: object

**Data Quality - Camada Silver:**
   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".
   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.
   - Estabeleça um sistema de monitoramento contínuo da qualidade dos dados da camada "silver".

##### calendar

In [63]:
calendar_raw_df.head(5)

,id_listing,data,disponibilidade,preco,preco_ajustado,minimo_noites,maximo_noites
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".

Verifique se há valores ausentes em cada coluna

In [64]:
valores_ausentes = calendar_raw_df.isnull().sum()
print(valores_ausentes)

id_listing         0
data               0
disponibilidade    0
preco              0
preco_ajustado     0
minimo_noites      3
maximo_noites      3
dtype: int64


Porcentagem de registros completos

In [65]:
total_registros = len(calendar_raw_df)
registros_completos = calendar_raw_df.dropna().shape[0]
porcentagem_completos = (registros_completos / total_registros) * 100

print(f'Porcentagem de registros completos: {porcentagem_completos:.2f}%')

Porcentagem de registros completos: 100.00%


   - Implemente verificações para garantir que os dados da camada "silver" estejam em conformidade com essas métricas.

Verifique se a coluna "disponibilidade" contém apenas os valores "t" ou "f"

In [66]:
valores_validos = ["t", "f"]
valores_incorretos = calendar_raw_df[-calendar_raw_df["disponibilidade"].isin(valores_validos)]

if not valores_incorretos.empty:
    print("Valores incorretos encontrados na coluna 'disponibilidade':")
    print(valores_incorretos)
else:
    print("Todos os valores na coluna 'disponibilidade' são válidos.")

Todos os valores na coluna 'disponibilidade' são válidos.


Verifique se há valores ausentes em colunas críticas

In [67]:
colunas_criticas = ['id_listing', 'data', 'disponibilidade', 'preco', 'preco_ajustado', 'minimo_noites', 'maximo_noites']
valores_ausentes = calendar_raw_df[colunas_criticas].isnull().sum()

if valores_ausentes.any():
    print("Valores ausentes encontrados nas colunas críticas:")
    print(valores_ausentes)
else:
    print("Nenhum valor ausente nas colunas críticas.")

Valores ausentes encontrados nas colunas críticas:
id_listing         0
data               0
disponibilidade    0
preco              0
preco_ajustado     0
minimo_noites      3
maximo_noites      3
dtype: int64


##### listings

In [68]:
listings_raw_df.head(5)

,id,nome,id_hospede,nome_hospede,grupo_bairro,bairro,latitude,longitude,tipo,preco,minimo_noites,numero_avaliacoes,ultima_avaliacao,avaliacoes_por_mes,numero_hospedes,disponibilidade_365,avaliacoes_ltm,licenca
0,17878,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,68997,Matthias,None,Copacabana,-22.96599,-43.17940,Entire home/apt,279,5,301,2023-09-11,1.87,1,265,25,None
1,25026,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,102840,Viviane,None,Copacabana,-22.97735,-43.19105,Entire home/apt,330,2,272,2023-09-07,1.68,1,203,24,None
2,35764,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,153691,Patricia Miranda & Paulo,None,Copacabana,-22.98107,-43.19136,Entire home/apt,192,3,446,2023-09-11,2.82,1,46,37,None
3,48305,Rental unit in Ipanema · ★4.74 · 6 bedrooms · ...,70933,Goitaca,None,Ipanema,-22.98591,-43.20302,Entire home/apt,3448,2,152,2023-09-10,0.99,9,306,30,None
4,48901,Rental unit in Rio · ★4.37 · 4 bedrooms · 5 be...,222884,Marcio,None,Copacabana,-22.96574,-43.17514,Entire home/apt,703,3,20,2023-09-10,0.20,1,307,12,None


   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".

Verifique se há valores ausentes em cada coluna

In [69]:
valores_ausentes = listings_raw_df.isnull().sum()
print(valores_ausentes)

id                         0
nome                       0
id_hospede                 0
nome_hospede               3
grupo_bairro           31964
bairro                     0
latitude                   0
longitude                  0
tipo                       0
preco                      0
minimo_noites              0
numero_avaliacoes          0
ultima_avaliacao        8140
avaliacoes_por_mes      8140
numero_hospedes            0
disponibilidade_365        0
avaliacoes_ltm             0
licenca                31964
dtype: int64


In [70]:
total_linhas = len(listings_raw_df.index)  
print(f'Número de linhas do DataFrame: {total_linhas}')

Número de linhas do DataFrame: 31964


Eliminação das linhas: grupo_bairro e licenca

In [71]:
columns_eliminate = ["grupo_bairro", "licenca"]
listings_raw_df = listings_raw_df.drop(columns_eliminate, axis=1)

Porcentagem de registros completos

In [72]:
total_registros = len(listings_raw_df)
registros_completos = listings_raw_df.dropna().shape[0]
porcentagem_completos = (registros_completos / total_registros) * 100

print(f'Porcentagem de registros completos: {porcentagem_completos:.2f}%')

Porcentagem de registros completos: 74.53%


##### reviews

In [73]:
reviews_raw_df.head(5)

,id_listing,id,data,reviewer_id,reviewer_name,comentarios
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...


   - Defina métricas de qualidade de dados, como integridade, precisão e consistência para os dados da camada "bronze".

Verifique se há valores ausentes em cada coluna

In [74]:
valores_ausentes = reviews_raw_df.isnull().sum()
print(valores_ausentes)

id_listing        0
id                0
data              0
reviewer_id       0
reviewer_name     0
comentarios      19
dtype: int64


In [75]:
total_linhas = len(reviews_raw_df.index)  
print(f'Número de linhas do DataFrame: {total_linhas}')

Número de linhas do DataFrame: 637307


Porcentagem de registros completos

In [76]:
total_registros = len(reviews_raw_df)
registros_completos = reviews_raw_df.dropna().shape[0]
porcentagem_completos = (registros_completos / total_registros) * 100

print(f'Porcentagem de registros completos: {porcentagem_completos:.2f}%')

Porcentagem de registros completos: 100.00%


Carregamento Processing Zone

calendar

In [77]:
danilo_freitas_processing_calendar = "danilo_freitas_processing_calendar"
trusted = "trusted"

In [78]:
db_params = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "ada_postgres_1009",
    "host": "database-ada-1009.clzgkgrmzmzh.us-east-1.rds.amazonaws.com",  
    "port": "5432"  
}

In [79]:
engine = create_engine(f'postgresql+psycopg2://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:{db_params["port"]}/{db_params["dbname"]}')

In [ ]:
calendar_raw_df.to_sql(danilo_freitas_processing_calendar, con=engine, schema=trusted, if_exists='replace', index=False)

listings

In [81]:
danilo_freitas_processing_listings = "danilo_freitas_processing_listings"
trusted = "trusted"

In [84]:
listings_raw_df.to_sql(danilo_freitas_processing_listings, con=engine, schema=trusted, if_exists='replace', index=False)

964

reviews

In [85]:
danilo_freitas_processing_reviews = "danilo_freitas_processing_reviews"
trusted = "trusted"

In [86]:
reviews_raw_df.to_sql(danilo_freitas_processing_reviews, con=engine, schema=trusted, if_exists='replace', index=False)

307

**Transformação de Dados com dbt - Camada Silver:**
   - Utilize a ferramenta dbt para criar a camada "silver" de dados, realizando transformações e preparando os dados da camada em questão.
   - Mantenha um controle de versão dos modelos dbt relacionados à camada "silver" e automatize a execução das transformações.

calendar

In [91]:
calendar_processing = "SELECT * FROM raw.danilo_freitas_raw_calendar;"

In [92]:
calendar_processing_df = pd.read_sql_query(calendar_processing, conn)

/tmp/ipykernel_26692/1883037361.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  calendar_processing_df = pd.read_sql_query(calendar_processing, conn)


In [93]:
calendar_processing_df.head(5)

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,17878,2023-09-23,f,$265.00,$265.00,5.0,28.0
1,17878,2023-09-24,t,$265.00,$265.00,5.0,28.0
2,17878,2023-09-25,f,$290.00,$290.00,5.0,28.0
3,17878,2023-09-26,f,$290.00,$290.00,5.0,28.0
4,17878,2023-09-27,f,$290.00,$290.00,5.0,28.0


listings

In [3]:
listings_processing = "SELECT * FROM trusted.danilo_freitas_processing_listings;"

In [8]:
listings_processing_df = pd.read_sql_query(listings_processing, conn)

/tmp/ipykernel_27787/1872804015.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  listings_processing_df = pd.read_sql_query(listings_processing, conn)


In [9]:
listings_processing_df.head(5)

,id,nome,id_hospede,nome_hospede,bairro,latitude,longitude,tipo,preco,minimo_noites,numero_avaliacoes,ultima_avaliacao,avaliacoes_por_mes,numero_hospedes,disponibilidade_365,avaliacoes_ltm
0,17878,Condo in Rio de Janeiro · ★4.70 · 2 bedrooms ·...,68997,Matthias,Copacabana,-22.96599,-43.17940,Entire home/apt,279,5,301,2023-09-11,1.87,1,265,25
1,25026,Rental unit in Rio de Janeiro · ★4.71 · 1 bedr...,102840,Viviane,Copacabana,-22.97735,-43.19105,Entire home/apt,330,2,272,2023-09-07,1.68,1,203,24
2,35764,Loft in Rio de Janeiro · ★4.90 · 1 bedroom · 1...,153691,Patricia Miranda & Paulo,Copacabana,-22.98107,-43.19136,Entire home/apt,192,3,446,2023-09-11,2.82,1,46,37
3,48305,Rental unit in Ipanema · ★4.74 · 6 bedrooms · ...,70933,Goitaca,Ipanema,-22.98591,-43.20302,Entire home/apt,3448,2,152,2023-09-10,0.99,9,306,30
4,48901,Rental unit in Rio · ★4.37 · 4 bedrooms · 5 be...,222884,Marcio,Copacabana,-22.96574,-43.17514,Entire home/apt,703,3,20,2023-09-10,0.20,1,307,12


reviews

In [10]:
reviews_processing = "SELECT * FROM trusted.danilo_freitas_processing_reviews;"

In [11]:
reviews_processing_df = pd.read_sql_query(reviews_processing, conn)

/tmp/ipykernel_27787/3087713142.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  reviews_processing_df = pd.read_sql_query(reviews_processing, conn)


In [12]:
reviews_processing_df.head(5)

,id_listing,id,data,reviewer_id,reviewer_name,comentarios
0,231497,726625,2011-11-17,1356783,Jenny,Es un apartastudio muy bien ubicado. Tiene lo ...
1,17878,64852,2010-07-15,135370,Tia,This apartment is in a perfect location -- two...
2,17878,76744,2010-08-11,10206,Mimi,we had a really great experience staying in Ma...
3,17878,91074,2010-09-06,80253,Jan,Staying in Max appartment is like living in a ...
4,17878,137528,2010-11-12,230449,Orene,In general very good and reasonable price.\r<b...
